In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scqubits as scq
# from quantiphy import Quantity
import pandas as pd
scq.settings.T1_DEFAULT_WARNING=False

In [ ]:
def I2flux(current, *flux_params):
    phi_zero, phi_half = flux_params
    phi = 2*np.abs(phi_half - phi_zero)
    if phi_zero > phi_half:
        phi_zero -= phi
    return (current-phi_zero)/phi

In [ ]:
df_csv = pd.read_csv("freqs.csv", usecols=['Current (mA)', 'Freq (MHz)', 'T1 (μs)'])
x = df_csv['Current (mA)']
fs = df_csv['Freq (MHz)']
y = df_csv['T1 (μs)']

# sort by current
x, fs, y = zip(*sorted(zip(x, fs, y), reverse=True))
x = np.array(x)
fs = np.array(fs)
y = np.array(y)
x_I = I2flux(x, 6.41926, -4.5606)

In [ ]:
fig, ax = plt.subplots(2,1,constrained_layout = True)
ax[0].set_title(r'$T_1$ as function of current')
ax[0].set_xlabel(r'Current (mA)')
ax[0].set_ylabel(r'$T_1$ ($\mu s$)')
ax[0].plot(x, y, '.-')

ax[1].set_title(r'$T_1$ as function of flux')
ax[1].set_xlabel(r'flux quanta $\phi_{ext}/\phi_0$')
ax[1].set_ylabel(r'$T_1$ ($\mu s$)')
ax[1].plot(x_I, y, '.-')
plt.show()

In [ ]:
fluxonium =  scq.Fluxonium(8.51, 0.91, 1.07, flux = 0.5, cutoff = 30, truncated_dim=5)
hilbertspace = scq.HilbertSpace([fluxonium])

In [ ]:
fig,ax = fluxonium.plot_coherence_vs_paramvals(
    param_name='flux',
    param_vals=np.linspace(0.5, 1.0, 501),
    common_noise_options=dict(i=1,j=0,T = 0.15),
    noise_channels = [('t1_capacitive', dict(Q_cap = 4e6))]
)
# ax.plot(x_I, y*1e3)

In [ ]:
Temp = 0.06
Q_ind = 7e7
Q_cap = 1e5
x_qp = 5e-5
flxs = np.linspace(0.5, 1.0, 501)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name='flux',
    param_vals=flxs,
    xlim=([0.49, 1.01]),
    common_noise_options=dict(i=1, j=0, T = Temp),
    noise_channels = [
                      # ('t1_inductive', dict(Q_ind = Q_ind)),
                      ('t1_quasiparticle_tunneling', dict(x_qp=x_qp)),
                      # ('t1_capacitive', dict(Q_cap = Q_cap))
                    ]
)
T1_eff_ax.plot(x_I, y*1e3, '.-')
T1_eff_ax.set_ylim([y.min()*5e2, y.max()*2e3])
_, T1_eff = T1_eff_ax.lines[0].get_data()
plt.show()
# T1up, Qcup = T1_eff, Q_cap
# T1down, Qcdown = T1_eff, Q_cap
# T1up, xqpup = T1_eff, x_qp
T1down, xqpdown = T1_eff, x_qp

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
# ax.plot(flxs, T1up*1e-9, label = f'Qc = {Qcup:.1e}')
# ax.plot(flxs, T1down*1e-9, label = f'Qc = {Qcdown:.1e}')
ax.plot(flxs, T1up*1e-9, label = f'xqp = {xqpup:.1e}')
ax.plot(flxs, T1down*1e-9, label = f'xqp = {xqpdown:.1e}')
ax.plot(x_I, y*1e-6, '.-', label = 'Data', color = 'purple')
ax.set_ylim([y.min()*5e-7, y.max()*2e-5])
ax.legend()
plt.show()

In [ ]:
flux_val = np.linspace(0.5, 1.0, 501)
T1_cap = []
T1_ind = []
T1_qp = []
for flux in flux_val:
    fluxonium.flux = flux
    T1_cap.append(fluxonium.t1_capacitive(T = Temp, Q_cap=Q_cap)*1e-9)
    T1_ind.append(fluxonium.t1_inductive(T = Temp, Q_ind=Q_ind)*1e-9)
    T1_qp.append(fluxonium.t1_quasiparticle_tunneling(T = Temp, x_qp=x_qp)*1e-9)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.plot(flux_val, T1_cap, label = 'Capacitive')
ax.plot(flux_val, T1_ind, label = 'Inductive')
ax.plot(flux_val, T1_qp, label = 'Quasi particle')
ax.plot(flux_val, T1_eff*1e-9, label = 'Effective')
ax.plot(x_I, y*1e-6, '.-', label = 'Data', color = 'purple')
ax.set_ylim([y.min()*5e-7, y.max()*2e-5])
ax.set_title(f'Q_cap = {Q_cap:.1e}, Q_ind = {Q_ind:.1e}, x_qp = {x_qp:.1e}')
ax.legend()
plt.show()